# Exploring the World Bank Data
Data can be found in [here](https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators)

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Explore").getOrCreate()
spark

In [124]:
base_path = "../../data/input/"
df = spark.read.csv(base_path + "WDIData.csv.bz2", header=True, inferSchema=True)
df.filter("`Country Code`='ARG'").limit(10).toPandas()
# df.show(3)
df.groupBy("`Country Code`").count().orderBy("`Country Code`").toPandas().head(4)
# df.columns

,Country Code,count
0,ABW,1486
1,AFE,1486
2,AFG,1486
3,AFW,1486


In [7]:
# df.printSchema()

## Some checkings of the content

In [125]:
# Starting some data
cols = [
    "Country Code",
    #"Country Name",
    "Indicator Code",
    #"Indicator Name",
    "2000", "2001"
]
# df1 = df.select(cols).filter("""
#     (`Country Code`='ARG' or `Country Code`like 'URY' or `Country Code`like 'CHL') and 
#     (`Indicator Code` like 'NY.ADJ.NNTY.PC.CD' or `Indicator Code` like 'EG.ELC.ACCS.ZS')
#         """).orderBy("Indicator Name")

df1 = df.select(cols).filter("""
    (`Country Code`='ARG') and 
    (`Indicator Code` like 'NY.ADJ.NNTY.PC.CD' or `Indicator Code` like 'EG.ELC.ACCS.ZS')
        """).orderBy("Indicator Name")

df1.show(10, 0)

+------------+-----------------+----------------+----------------+
|Country Code|Indicator Code   |2000            |2001            |
+------------+-----------------+----------------+----------------+
|ARG         |EG.ELC.ACCS.ZS   |95.6804733276367|95.5110634674458|
|ARG         |NY.ADJ.NNTY.PC.CD|6430.93331567539|6002.28904723084|
+------------+-----------------+----------------+----------------+



In [44]:
df1.printSchema()

root
 |-- Country Code: string (nullable = true)
 |-- Indicator Code: string (nullable = true)
 |-- 2000: double (nullable = true)
 |-- 2001: double (nullable = true)



In [57]:
import pyspark.sql.functions as F
(df1
 .select(
     "Country Code", 
     "Indicator Code",
     F.create_map(
         F.lit("2000"), F.col("2000"),
         F.lit("2001"), F.col("2001")
     ).alias("values")
).show(10, 0)

+------------+-----------------+----------------------------------------------------+
|Country Code|Indicator Code   |values                                              |
+------------+-----------------+----------------------------------------------------+
|ARG         |EG.ELC.ACCS.ZS   |{2000 -> 95.6804733276367, 2001 -> 95.5110634674458}|
|ARG         |NY.ADJ.NNTY.PC.CD|{2000 -> 6430.93331567539, 2001 -> 6002.28904723084}|
+------------+-----------------+----------------------------------------------------+



In [100]:
import pyspark.sql.functions as F
df2 = (df1
 .select(
     "Country Code", 
     "Indicator Code",
     F.create_map(
         F.lit("2000"), F.col("2000"),
         F.lit("2001"), F.col("2001")
     ).alias("values")
 ).select("Country Code", "Indicator Code", F.explode("values")).withColumnRenamed('key', 'year')
)

df2.show(10, 0)
df2.printSchema()

+------------+-----------------+----+----------------+
|Country Code|Indicator Code   |year|value           |
+------------+-----------------+----+----------------+
|ARG         |EG.ELC.ACCS.ZS   |2000|95.6804733276367|
|ARG         |EG.ELC.ACCS.ZS   |2001|95.5110634674458|
|ARG         |NY.ADJ.NNTY.PC.CD|2000|6430.93331567539|
|ARG         |NY.ADJ.NNTY.PC.CD|2001|6002.28904723084|
+------------+-----------------+----+----------------+

root
 |-- Country Code: string (nullable = true)
 |-- Indicator Code: string (nullable = true)
 |-- year: string (nullable = false)
 |-- value: double (nullable = true)



Now, automating the columns creation

In [107]:
def map_content(df):
    columns = df.columns
    r = []
    for c in columns:
        # we want to keep only the columns with a number (year) in its name.
        if c.isnumeric():
            r.append(F.lit(str(c)))
            r.append(F.col(str(c)))
    return r
    
map_content(df1)

[Column<'2000'>, Column<'2000'>, Column<'2001'>, Column<'2001'>]

In [109]:
df2 = (df1
 .select(
     "Country Code", 
     "Indicator Code",
     F.create_map(*map_content(df1)).alias("values")
 ).select("Country Code", "Indicator Code", F.explode("values")).withColumnRenamed('key', 'year')
)

df2.show(10, 0)
df2.printSchema()

+------------+-----------------+----+----------------+
|Country Code|Indicator Code   |year|value           |
+------------+-----------------+----+----------------+
|ARG         |EG.ELC.ACCS.ZS   |2000|95.6804733276367|
|ARG         |EG.ELC.ACCS.ZS   |2001|95.5110634674458|
|ARG         |NY.ADJ.NNTY.PC.CD|2000|6430.93331567539|
|ARG         |NY.ADJ.NNTY.PC.CD|2001|6002.28904723084|
+------------+-----------------+----+----------------+

root
 |-- Country Code: string (nullable = true)
 |-- Indicator Code: string (nullable = true)
 |-- year: string (nullable = false)
 |-- value: double (nullable = true)



## Checking some of the countries

In [126]:
df1 = df.filter("(`Country Code`='ARG' or `Country Code`like 'URY' or `Country Code`like 'CHL') and `Indicator Code`='NY.ADJ.NNTY.PC.CD'")
df1.select("Country Name", "`Country Code`", "`Indicator Code`", "`Indicator Name`").show(300, 0)

+------------+------------+-----------------+-----------------------------------------------------+
|Country Name|Country Code|Indicator Code   |Indicator Name                                       |
+------------+------------+-----------------+-----------------------------------------------------+
|Argentina   |ARG         |NY.ADJ.NNTY.PC.CD|Adjusted net national income per capita (current US$)|
|Chile       |CHL         |NY.ADJ.NNTY.PC.CD|Adjusted net national income per capita (current US$)|
|Uruguay     |URY         |NY.ADJ.NNTY.PC.CD|Adjusted net national income per capita (current US$)|
+------------+------------+-----------------+-----------------------------------------------------+



In [127]:
df1.limit(10).toPandas()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,_c67
0,Argentina,ARG,Adjusted net national income per capita (curre...,NY.ADJ.NNTY.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,10265.670658,11761.781230,10865.789888,12578.146137,9879.584610,8244.068678,7211.541026,8993.547410,NaN,None
1,Chile,CHL,Adjusted net national income per capita (curre...,NY.ADJ.NNTY.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,11688.278247,11281.813072,11133.063386,11819.491877,12167.308446,11556.470339,9756.161448,11095.880284,NaN,None
2,Uruguay,URY,Adjusted net national income per capita (curre...,NY.ADJ.NNTY.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,14854.510424,14048.435558,14949.304683,16557.208784,16444.154738,15768.940291,13661.846748,14591.582026,NaN,None


In [142]:
# df.select("Indicator Name").distinct().show(20, 0)
df.select("Indicator Code", "Indicator Name").where("`Indicator Name` like '%poverty%'").distinct().show(200, 0)

+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------+
|Indicator Code   |Indicator Name                                                                                                                               |
+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------+
|SI.POV.MDIM.HH   |Multidimensional poverty headcount ratio, household (% of total households)                                                                  |
|SI.POV.MDIM      |Multidimensional poverty headcount ratio (% of total population)                                                                             |
|SH.UHC.TOT2.ZS   |Proportion of population pushed or further pushed below the $3.65 ($ 2017 PPP) poverty line by out-of-pocket health care expenditure (%)     |
|SH.UHC.TOTR.ZS   |Proportio

In [146]:
df.createOrReplaceTempView("WDIData")
spark.sql("""
    SELECT
        `Indicator Code`, count(*)
    FROM
        WDIData
    WHERE
        `Country Code` = 'ARG'
        AND `Indicator Name` like '%poverty%'
    GROUP BY `Indicator Code`
    ORDER BY 2 desc
""").show(20, 0)

+-----------------+--------+
|Indicator Code   |count(1)|
+-----------------+--------+
|SI.POV.MDIM.FE   |1       |
|SH.UHC.TOTR.ZS   |1       |
|SH.UHC.TOT1.ZS   |1       |
|SE.LPV.PRIM.FE   |1       |
|SI.POV.MDIM.17   |1       |
|SI.POV.MDIM.HH   |1       |
|SH.UHC.FBP1.ZS   |1       |
|SI.POV.MDIM.MA   |1       |
|SI.POV.MDIM.XQ   |1       |
|SH.UHC.NOPR.ZS   |1       |
|SI.POV.MDIM      |1       |
|SE.LPV.PRIM.MA   |1       |
|SH.UHC.FBP2.ZS   |1       |
|SI.POV.NAHC      |1       |
|SH.UHC.TOT2.ZS   |1       |
|SI.POV.MDIM.IT   |1       |
|SH.UHC.NOP2.ZS   |1       |
|SE.LPV.PRIM      |1       |
|SH.UHC.NOP1.ZS   |1       |
|SI.POV.MDIM.17.XQ|1       |
+-----------------+--------+
only showing top 20 rows



## What about the rest of the files ?

In [113]:
!ls -l ../../data/input/

total 114384
-rw-r--r--  1 facundoradrizzani  staff   1053933 Dec 18 23:06 WDICountry-Series.csv
-rw-r--r--  1 facundoradrizzani  staff    157219 Dec 18 23:06 WDICountry.csv
-rw-r--r--  1 facundoradrizzani  staff  53327735 Dec 18 23:06 WDIData.csv.bz2
-rw-r--r--  1 facundoradrizzani  staff     14917 Dec 18 23:06 WDISeries-Time.csv
-rw-r--r--  1 facundoradrizzani  staff   3999671 Dec 18 23:06 WDISeries.csv


In [121]:
# WDICountry creates a relation with groups for each country such as Income Groupe or Region, very useful for anlytics.
WDICountry = spark.read.csv(base_path + "WDICountry.csv", header=True, inferSchema=True)
WDICountry.select("Country Code", "Short Name", "Region", "Income Group", "Latest population census", "Latest trade data").where("Region like '%Latin%'").orderBy("Short Name").show(20)

+------------+--------------------+--------------------+-------------------+------------------------+-----------------+
|Country Code|          Short Name|              Region|       Income Group|Latest population census|Latest trade data|
+------------+--------------------+--------------------+-------------------+------------------------+-----------------+
|         ATG| Antigua and Barbuda|Latin America & C...|        High income|                    2011|             2018|
|         ARG|           Argentina|Latin America & C...|Upper middle income|         2020 (expected)|             2018|
|         ABW|               Aruba|Latin America & C...|        High income|         2020 (expected)|             2018|
|         BRB|            Barbados|Latin America & C...|        High income|         2020 (expected)|             2018|
|         BLZ|              Belize|Latin America & C...|Upper middle income|         2020 (expected)|             2018|
|         BOL|             Bolivia|Latin